In [224]:
import argparse
import os
import glob
import pandas as pd
import json
from utils import load_json

In [225]:
f_json = load_json("필터 결과.json")
df_check = pd.read_csv("../meta/checklist.csv")
df_data = pd.DataFrame([load_json(d) for d in glob.glob(f'extract/*.json')])

In [226]:
korq = {'data':[
            {'title':8,
            }]
        }
if 'doc_id' in f_json:
    del f_json['doc_id']

req_qid = {}
for k, vs in f_json.items():
    for v in vs:
        v = v.split('_')[-1].split('.')[0]
        req_qid[v] = req_qid.get(v, []) + [k]

In [227]:
para = []
for req, qid in req_qid.items():
    context = {'context_id':req}
    req_data = df_data[df_data.pid==req]
    context['context'] = req_data.context.values[0]
    context['qas'] = []
    for q in qid:
        qa = {}
        qa['id'] = f'8_{req}_{q}'
        qa['question'] = df_check[df_check.question_id==q].sub_question.values[0]
        qa['answers'] = []
        tmp = pd.DataFrame(req_data.answers.values[0])
        tmp = tmp[(tmp.question_id==int(q.split('_')[0])) & (tmp.sub_question_id==int(q.split('_')[1]))]
        if tmp.answer.values[0]==0:
            qa["is_impossible"] = True
        else:
            qa["is_impossible"] = False
            ans = {}
            ans['text'] = tmp.extract.values[0]
            ans['answer_start'] = req_data.context.values[0].index(tmp.extract.values[0])
            qa['answers'].append(ans)
        context['qas'].append(qa)
    para.append(context)
korq['data'][0]['paragraphs'] = para

In [251]:
for data in korq['data']:
    for para in data['paragraphs']:
        if para['context_id']=='DAR-001':
            print(para['context'])

정의: 고용행정포털 데이터베이스 모델링 설계

세부 내용 : ㅇ DB 구조의 설계는 관련 업무 처리 절차를 반영하여 유기적으로 구조화하고 향후 업무 변동에 따른 확장성을 충분히 고려해야 함
ㅇ 데이터 관리 지침을 수용할 수 있도록 DB설계가 되어야 함
ㅇ 데이터의 정합성을 유지하면서 시스템의 성능을 저하시키지 않도록 DB 설계가 되어야함.
ㅇ 주관기관의 데이터 품질 가이드와 행정정보 데이터 베이스 표준화 지침을 준수하여 DB를 설계하여야 함 
ㅇ DB설계는 모델링 도구를 활용하여 작성하고 공유 할 수 있어야 함
ㅇ DB세부항목 정합성 심층 분석 및 정제
  - 정합성 규칙 작성, 데이터 분석·정제·관리 개선 방안 도출
ㅇ 데이터 모델링
  - 현행 업무시스템을 분석하여 데이터 관련 요건 및 미사용 테이블, 최종 데이터생성 시기 등을 분석하여 데이터베이스 설계 표준 및 가이드에 따라서 주제영역 정의, 개념적, 논리적, 물리적 데이터 모델링을 수행
  - 개별적으로 설계되어 요건변경에 취약한 구조의 데이터베이스를 통합·체계화하는 등 기본구조가 명확한 개념모델로 설계하여 논리 데이터모델 전개 방향성 제시
  - 업무특성 및 연관성을 고려하여 중복이 발생하지 않고, 공유·활용 및 업무요건변경, 프로그램 수정요청에 따른 구조적 변화가 최소화 될 수 있도록 유연한 통합 DB설계 (논리/물리모델)
  - 시스템의 기준정보를 정의하고, 주체와 행위를 명확하게 구분하여 응용시스템 구조 변경과 업무 변화까지 고려하여 설계
  - 대량 업무정보 조회와 탐색 등에도 데이터베이스의 성능이 확보될 수 있도록 설계
  - 데이터 표준화규칙에 기반한 명명규칙에 따라 논리모델 및 물리모델의 데이터 요소에 반영
  - 개인정보가 노출되지 않도록 보안을 고려
  - 데이터 모델링 수행 시 그간 고용24 구축방안 수립 ISP 컨설팅 결과를 기반으로 구체적인 모델을 도출하여야 함
ㅇ 데이터 모델링 실행 방안 제시
  - 데이터 모델링에 대한 방법론, 투입 인력, 기간, 상세 수행 내역 등 종합적으

In [239]:
save_name = f"korquality_40.json"
with open(os.path.join("./", save_name), "w", encoding="utf-8") as json_file:
    json.dump(korq, json_file, ensure_ascii=False, indent="\t")

In [240]:
raw = []
for para in korq['data'][0]['paragraphs']:
    for qas in para['qas']:
        ans = qas['answers'][0]['text'] if qas['answers'] else ""
        raw.append([para['context_id'], qas['id'], ans, qas['is_impossible']])

In [241]:
raw = pd.DataFrame(raw, columns=['요구사항', 'id', '판별근거', 'is_impossible'])

In [242]:
raw['qid'] = raw.id.apply(lambda x: int(x.split('_')[2]))
raw['sub_qid'] = raw.id.apply(lambda x: int(x.split('_')[3]))

In [243]:
raw = raw.sort_values(['qid','sub_qid','요구사항']).reset_index(drop=True)

In [244]:
raw['반영여부'] = raw.is_impossible.apply(lambda x: 0 if x else 1)

In [245]:
raw[['qid','sub_qid','요구사항','반영여부','판별근거']].to_csv('시연용데이터GT.csv', encoding='cp949', index=False)

In [246]:
pr = load_json("../../data/predictions_final.json")

In [247]:
pr = pd.DataFrame(pr.items(), columns =['id','모델예측'])

In [248]:
pd.merge(raw, pr)[['qid','sub_qid','요구사항','반영여부','판별근거','모델예측']]

,qid,sub_qid,요구사항,반영여부,판별근거,모델예측
0,1,1,DAR-002,1,"- 코드관리체계, 표준화(단어, 용어, 도메인 등) 방안 등 수립",-
1,1,1,DAR-004,0,,- 발주기관의 표준도메인을 준수하고 표준도메인 기준으로 표준용어를 활용 및 생성하여야 함
2,1,1,DAR-012,0,,
3,2,1,DAR-002,1,"- 시스템 구축에 따라 추가가 필요한 데이터 표준항목(단어, 용어, 도메인)을 정의...","공통표준용어(행정안전부고시), 공공데이터 관리지침(행정안전부고시), 공공데이터 품질..."
4,2,1,DAR-003,0,,
...,...,...,...,...,...,...
208,19,3,DAR-016,0,,*
209,19,3,DAR-019,1,- 본 사업을 통해 구축되는 시스템과 연관된 개방데이터 목록을 식별하고 개방데이...,-
210,20,1,DAR-004,0,,
211,20,1,DAR-007,0,,
